In [1]:
import requests
import base64
import os
import json
from flask import Flask
import urllib.parse 
import pandas as pd
from IPython.display import Image, display, HTML

https://developer.spotify.com/documentation/web-api/tutorials/getting-started

In [2]:
def get_spotify_access_token(client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()
    }
    data = {
        'grant_type': 'client_credentials'
    }
    
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        token_data = response.json()
        
        if 'access_token' in token_data:
            return token_data['access_token']
        else:
            raise Exception(f"Errore nell'ottenere il token di accesso: {token_data.get('error', 'Errore sconosciuto')}")
            
    except requests.exceptions.RequestException as e:
        raise Exception(f"Errore nella richiesta HTTP: {e}")

In [3]:
def search_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        artists = data.get("artists", {}).get("items", [])
        
        if artists:
            return artists[0]
        else:
            print(f"Nessun artista trovato con il nome '{artist_name}'.")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return None


In [4]:
def get_albums(token, artist_id):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums'
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "market": "IT"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        
        return data.get("items", [])
        
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return []

In [5]:
def get_album_tracks(token, album_id):
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "market": "IT"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        
        return data.get("items", [])
        
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return []

In [6]:
def top_tracks_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=IT"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.content)["tracks"]
        return data
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  

## Testing delle funzioni realizzate

In [7]:
client_id = 'cd0f7e9279034469a5c4560808cb31a8' 
client_secret = 'da3fe74efc5c4a84b71399c157366e09'
access_token = get_spotify_access_token(client_id, client_secret) # ottengo il token di accesso

print(f"Token di accesso: {access_token}")

Token di accesso: BQCU_YQR8qFCYOARGmoqJa2yaa24pyxA_54tpU1Mpi8V3MdqQkR8fj8QBIImOudrcm4VvVclGPl9zuAVhpzqql6Y9rlhcGgqW6Srf0wIQ175ixo2lo8


In [8]:
artist_name = "Emis killa" 
artist = search_artist(access_token, artist_name) # ottengo l'artista

artist_data = {
    "Name": artist["name"],
    "ID": artist["id"] ,
    "Spotify URL": artist["external_urls"]["spotify"],
    "Genere": artist["genres"],
    "Followers": artist["followers"]["total"],
    "Artist image URL": artist["images"][0]["url"]
}

print(f"Artista trovato: {artist_data['Name']} (ID: {artist_data['ID']})\n") 

Artista trovato: Emis Killa (ID: 6FtwCmLY6L1sqvjaQ2lV6G)



In [9]:
artist_albums = get_albums(access_token, artist_data['ID']) # ottengo gli album dell'artista

data = []
for album in artist_albums:
    data.append({
        "Nome Album": album['name'],
        "ID Album": album['id'],
        "Data di Uscita": album['release_date'],
        "Tipo Album": album['album_type'],
        "Numero Totale di Tracce": album['total_tracks'],
        "URI Album": album['uri']
    })

album_data = pd.DataFrame(data, columns=['ID Album', 'Nome Album', 'Data di Uscita', 'Tipo Album', 'Numero Totale di Tracce', 'URI Album'])

print(f"Gli album dell'artista {artist_name} sono:\n")
album_data

Gli album dell'artista Emis killa sono:



,ID Album,Nome Album,Data di Uscita,Tipo Album,Numero Totale di Tracce,URI Album
0,39wuRno08Rl0Jsse2kCHga,Effetto notte,2023-05-19,album,14,spotify:album:39wuRno08Rl0Jsse2kCHga
1,53qzcXW1K4TdnJZUp0Ftpu,L'erba cattiva (10 Years Anniversary Edition),2022-12-09,album,15,spotify:album:53qzcXW1K4TdnJZUp0Ftpu
2,5XmLyPktaR9fYxYqtQukqa,"Keta Music, Vol. 3",2021-07-23,album,11,spotify:album:5XmLyPktaR9fYxYqtQukqa
3,6nKQCH9bP1APHf83iDCjO2,17 - Dark Edition,2021-02-26,album,24,spotify:album:6nKQCH9bP1APHf83iDCjO2
4,11T8pkk7VAT7LeoReyY7B7,17,2020-09-18,album,17,spotify:album:11T8pkk7VAT7LeoReyY7B7
5,3YdgMr7sbDBeuPotzWJz60,Supereroe Bat Edition,2019-06-21,album,17,spotify:album:3YdgMr7sbDBeuPotzWJz60
6,67g3eUjlYFB7EkhS1pjKe4,Supereroe,2018-10-12,album,13,spotify:album:67g3eUjlYFB7EkhS1pjKe4
7,2goDKoHetJSDYCxZq9h17h,Terza Stagione,2016-10-14,album,17,spotify:album:2goDKoHetJSDYCxZq9h17h
8,2P8Ga6LVlc6lwuKwxsrbsd,Keta Music - Volume 2,2015-06-18,album,13,spotify:album:2P8Ga6LVlc6lwuKwxsrbsd
9,5ZPw8Dc0rcfLBlJj9zJ9pK,Mercurio - 5 Stars Edition,2014-06-10,album,19,spotify:album:5ZPw8Dc0rcfLBlJj9zJ9pK


In [10]:
album_name = "Effetto notte"  
album_id = album_data.loc[album_data['Nome Album'] == album_name, 'ID Album'].values[0]
album_tracks = get_album_tracks(access_token, album_id)

print(f"Le tracce dell'album {album_name} sono:\n")
for idx,track in enumerate (album_tracks):
    print(f"{idx+1}) {track['name']}, Durata: {track['duration_ms']/60000:.2f} minuti")

artist_data

Le tracce dell'album Effetto notte sono:

1) PACINO - the godfather, Durata: 2.04 minuti
2) ON FIRE - paid in full, Durata: 2.93 minuti
3) COL CUORE IN GOLA - lords of dogtown, Durata: 3.14 minuti
4) VISCERALE - closer, Durata: 3.07 minuti
5) MASERATI - the wolf of wall street, Durata: 2.27 minuti
6) LONTANO - carlito's way, Durata: 2.78 minuti
7) ALBICOCCA - lolita, Durata: 2.71 minuti
8) BIG BANG - upside down, Durata: 3.30 minuti
9) TOXIC - trainspotting, Durata: 3.00 minuti
10) SONNY - city of god, Durata: 3.18 minuti
11) MC DRIVE - la haine, Durata: 3.00 minuti
12) SENZ'ANIMA - nikita, Durata: 3.22 minuti
13) ATTORI DI STRADA - ragazzi fuori, Durata: 3.51 minuti
14) BEL FINALE - the butterfly effect, Durata: 1.86 minuti


{'Name': 'Emis Killa',
 'ID': '6FtwCmLY6L1sqvjaQ2lV6G',
 'Spotify URL': 'https://open.spotify.com/artist/6FtwCmLY6L1sqvjaQ2lV6G',
 'Genere': ['italian hip hop', 'italian pop'],
 'Followers': 1545181,
 'Artist image URL': 'https://i.scdn.co/image/ab6761610000e5eb51507026f810de5f84d9e8a2'}

In [11]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([artist_data])

# Visualizza l'immagine centrata
image_html = f'<div style="display: flex; justify-content: center;">' \
             f'<img src="{df["Artist image URL"][0]}" alt="Artista" width="{250}" style="display: block;"/>' \
             f'</div>'
display(HTML(image_html))

#df drop rimuove la collone Artist image URL e visualizza il dataframe
display(df.drop("Artist image URL", axis=1))

,Name,ID,Spotify URL,Genere,Followers
0,Emis Killa,6FtwCmLY6L1sqvjaQ2lV6G,https://open.spotify.com/artist/6FtwCmLY6L1sqvjaQ2lV6G,"[italian hip hop, italian pop]",1545181


In [14]:
from flask import Flask, redirect, request, url_for
import random
import string
import urllib.parse

app = Flask(__name__)

# Definisci il tuo client_id e redirect_uri
redirect_uri = 'http://localhost:5000/callback'

# Funzione per generare una stringa casuale
def generate_random_string(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for _ in range(length))

@app.route('/login')
def login():
    state = generate_random_string(16)
    scope = 'user-read-private user-read-email'
    params = {
        'response_type': 'code',
        'client_id': client_id,
        'scope': scope,
        'redirect_uri': redirect_uri,
        'show_dialog': True,
        'state': state
    }
    query_string = urllib.parse.urlencode(params)
    return redirect('https://accounts.spotify.com/authorize?' + query_string)

if __name__ == '__main__':
    app.run(debug=True, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nameisalfio/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1052, in launch_instance
    app.initialize(argv)
  File "/home/nameisalfio/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 117, in inner
    return method(app, *args, **kwargs)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_soc

SystemExit: 1